In [1]:
'''--------------------导入MindSpore模块和辅助模块--------------------'''
import csv
import os
import time
import numpy as np
from easydict import EasyDict as edict
from matplotlib import pyplot as plt
import mindspore
from mindspore import nn
from mindspore import context
from mindspore import dataset
from mindspore.train.callback import TimeMonitor, LossMonitor
from mindspore import Tensor
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig # 回调机制
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend") # 设定运行模式为静态图模式，并且运行设备为昇腾芯片

In [2]:
'''--------------------模型参数设置--------------------'''
cfg = edict({
    'data_size': 150,
    'train_size': 120, #训练集大小
    'test_size': 30 , #测试集大小
    'feature_number': 4, #输入特征数
    'num_class': 3, #分类类别
    'batch_size': 30, #批次大小
    'data_dir': 'iris.data', # 数据集路径
    'save_checkpoint_steps': 5, #多少步保存一次模型
    'keep_checkpoint_max': 1, #最多保存多少个模型
    'out_dir_no_opt': './model_iris/no_opt', #保存模型路径，无优化器模型
    'out_dir_sgd': './model_iris/sgd', #保存模型路径,SGD优化器模型
    'out_dir_momentum': './model_iris/momentum', #保存模型路径，momentum模型
    'out_dir_adam': './model_iris/adam', #保存模型路径，adam优化器模型
    'output_prefix': "checkpoint_fashion_forward" #保存模型文件名
})

In [6]:
'''--------------------读取数据并根据MindSpore模型需求进行预处理--------------------
共150条数据,将数据集的4个属性作为自变量X,3个类别映射为{0,1,2},作为因变量Y。'''
#鸢尾花数据集，本数据集共有150个带标签的数据
with open(cfg.data_dir) as csv_file:
    data = list(csv.reader(csv_file, delimiter=','))

label_map = {'setosa': 0,'versicolor': 1,'virginica':2 }
#分别获取数据中的特征值X和标签值Y
X = np.array([[float(x) for x in s[:-1]] for s in data[:cfg.data_size]],np.float32)
Y = np.array([label_map[s[-1]] for s in data[:cfg.data_size]], np.int32)

# 将数据集分为训练集120条，测试集30条。
train_idx = np.random.choice(cfg.data_size, cfg.train_size, replace=False)
test_idx = np.array(list(set(range(cfg.data_size)) - set(train_idx)))

X_train, Y_train = X[train_idx], Y[train_idx]
X_test, Y_test = X[test_idx], Y[test_idx]

In [32]:
'''--------------------使用MindSpore GeneratorDataset接口将numpy.ndarray类型的数据转换为Dataset--------------------'''
XY_train = list(zip(X_train, Y_train))
train_dataset = dataset.GeneratorDataset(XY_train, ['x', 'y'])
train_dataset = train_dataset.shuffle(buffer_size=cfg.train_size).batch(cfg.batch_size, drop_remainder=True)
XY_test = list(zip(X_test, Y_test))
test_dataset = dataset.GeneratorDataset(XY_test, ['x', 'y'])
test_dataset = test_dataset.shuffle(buffer_size=cfg.test_size).batch(cfg.test_size, drop_remainder=True)

In [33]:
'''--------------------Adam优化器--------------------'''
#定义神经网络、优化器和损失函数供模型Model使用
Network = nn.Dense(cfg.feature_number, cfg.num_class)
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
optimizer=nn.Adam(Network.trainable_params())
#调用模型Model，使用优化器，评价函数采用准确率
model = Model(network=Network, loss_fn=net_loss, optimizer=optimizer, metrics={"Acc":nn.Accuracy()})

# 定义回调类用来输出训练过程的信息
config_ck = CheckpointConfig(save_checkpoint_steps=cfg.save_checkpoint_steps,keep_checkpoint_max=cfg.keep_checkpoint_max)
ckpt_cb = ModelCheckpoint(prefix="checkpoint_no_opt", directory= cfg.out_dir_no_opt, config=config_ck)
loss_cb = LossMonitor(400)

#训练
print('--------------------Adam优化器--------------------')
print('==========Start Training==========')
model.train(epoch=500,train_dataset=train_dataset, callbacks=[ckpt_cb, loss_cb])

# 模型评估
Acc=model.eval(test_dataset)
print(Acc)

# 输出预测结果
test_iter = test_dataset.create_dict_iterator()._get_next() # 构造迭代器
features = Tensor(test_iter['x'], mindspore.float32) # 构造特征张量
predictions = model.predict(features) # 预测结果
predictions = predictions.asnumpy() #转换成numpy格式
num2label=['Setosa','Versicolor','Virginica']
for i in range(30):
    sample = predictions[i,:].tolist()
    print('第'+str(i)+'个sample预测结果: ',num2label[sample.index(max(sample))],'    真实结果: ', num2label[test_iter['y'][i]])

--------------------Adam优化器--------------------
==========Start Training==========
epoch: 100 step: 4, loss is 0.6743768453598022
epoch: 200 step: 4, loss is 0.5596011877059937
epoch: 300 step: 4, loss is 0.4155047535896301
epoch: 400 step: 4, loss is 0.32300201058387756
epoch: 500 step: 4, loss is 0.3883627653121948
{'Acc': 1.0}
第0个sample预测结果:  Setosa     真实结果:  Setosa
第1个sample预测结果:  Setosa     真实结果:  Setosa
第2个sample预测结果:  Setosa     真实结果:  Setosa
第3个sample预测结果:  Virginica     真实结果:  Virginica
第4个sample预测结果:  Setosa     真实结果:  Setosa
第5个sample预测结果:  Setosa     真实结果:  Setosa
第6个sample预测结果:  Setosa     真实结果:  Setosa
第7个sample预测结果:  Virginica     真实结果:  Virginica
第8个sample预测结果:  Setosa     真实结果:  Setosa
第9个sample预测结果:  Versicolor     真实结果:  Versicolor
第10个sample预测结果:  Virginica     真实结果:  Virginica
第11个sample预测结果:  Versicolor     真实结果:  Versicolor
第12个sample预测结果:  Versicolor     真实结果:  Versicolor
第13个sample预测结果:  Versicolor     真实结果:  Versicolor
第14个sample预测结果:  Setosa     真实结果:  Setos